In [8]:
#Import Dependancies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the requests library.
import requests
#Import the datetime module from the datetime library
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the API key.
from config import weather_api_key
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [20]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = list(zip(lats, lngs))
lat_lngs
# Add the latitudes and longitudes to a list.
coordinates = lat_lngs

In [23]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


616

In [24]:
#Create empty list to hold the weather data
city_data = []
#print the beginning of the logging
print("Beginning Data Retrieval       ")
print("---------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval       
---------------------------------
Processing Record 1 of Set 1 | cherskiy
Processing Record 2 of Set 1 | ancona
Processing Record 3 of Set 1 | provideniya
Processing Record 4 of Set 1 | oranjestad
Processing Record 5 of Set 1 | aklavik
Processing Record 6 of Set 1 | vaini
Processing Record 7 of Set 1 | busselton
Processing Record 8 of Set 1 | koumac
Processing Record 9 of Set 1 | hobart
Processing Record 10 of Set 1 | bengkulu
Processing Record 11 of Set 1 | ushuaia
Processing Record 12 of Set 1 | midland
Processing Record 13 of Set 1 | sur
Processing Record 14 of Set 1 | avarua
Processing Record 15 of Set 1 | souillac
Processing Record 16 of Set 1 | albany
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | caceres
Processing Record 19 of Set 1 | jamestown
Processing Record 20 of Set 1 | zachary
Processing Record 21 of Set 1 | vestmannaeyjar
Processing Record 22 of Set 1 | takoradi
Processing Record 23 of Set 1 | arraial do cabo
Proc

In [34]:
#Convert the array of dictionaries to a pandas df
city_data_df = pd.DataFrame(city_data)

new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed" ]
city_data_df = city_data_df[new_column_order]

#Create an output file (CSV)
output_data_file = "weather_data/cities.csv"
#Export data to CSV
city_data_df.to_csv(output_data_file, index_label = "City ID")